### Get the webpage using *requests* module

In [28]:
import requests
response = requests.get('https://en.wikipedia.org/wiki/Set_(mathematics)') 

# response type
print(type(response))
print()
# type from .text method of requests.get
print(type(response.text))
print()
# print actual HTML code
print(response.text)


<class 'requests.models.Response'>

<class 'str'>

<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>Set (mathematics) - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Set_(mathematics)","wgTitle":"Set (mathematics)","wgCurRevisionId":875581816,"wgRevisionId":875581816,"wgArticleId":26691,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 maint: Archived copy as title","Wikipedia articles with GND identifiers","Concepts in logic","Mathematical objects","Set theory"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTrans

### Messy HTML

### Import BeautifulSoup from *bs4*  module and test

In [29]:
from bs4 import BeautifulSoup
# create some sample HTML

html_doc = """
<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>

<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>

<p class="story">...</p>
"""

soup = BeautifulSoup(html_doc, 'html.parser')

In [30]:
# make the HTML readable
print(soup.prettify())

<html>
 <head>
  <title>
   The Dormouse's story
  </title>
 </head>
 <body>
  <p class="title">
   <b>
    The Dormouse's story
   </b>
  </p>
  <p class="story">
   Once upon a time there were three little sisters; and their names were
   <a class="sister" href="http://example.com/elsie" id="link1">
    Elsie
   </a>
   ,
   <a class="sister" href="http://example.com/lacie" id="link2">
    Lacie
   </a>
   and
   <a class="sister" href="http://example.com/tillie" id="link3">
    Tillie
   </a>
   ;
and they lived at the bottom of a well.
  </p>
  <p class="story">
   ...
  </p>
 </body>
</html>


### Make a BeautifulSoup object from webpage HTML

In [31]:
# create a soup from the webpage
soup2 = BeautifulSoup(response.text, 'html.parser')
# print a cleaned up version of the webpage HTML
print(soup2.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Set (mathematics) - Wikipedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Set_(mathematics)","wgTitle":"Set (mathematics)","wgCurRevisionId":875581816,"wgRevisionId":875581816,"wgArticleId":26691,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 maint: Archived copy as title","Wikipedia articles with GND identifiers","Concepts in logic","Mathematical objects","Set theory"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDe

In [ ]:
type(response.text)

### Test function to find first link

In [32]:
def find_first_link(url):
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, "html.parser")

    # This div contains the article's body
    # (Note: Body nested in two div tags)
    content_div = soup.find(id="mw-content-text").find(class_="mw-parser-output")
        
    # stores the first link found in the article, if the article contains no
    # links this value will remain None
    article_link = None

    for element in content_div.find_all("p", recursive=False):
        if element.find("a", recursive=False) :
            article_link = element.find("a", recursive=False).get('href')
            if article_link.find("Help") <0:
                break
            print(article_link)
    # find the first link in the article, or set to None if
    # there is no link in the article.
    

    if not article_link:
        return

    # Build a full url from the relative article_link url
    first_link = urllib.parse.urljoin('https://en.wikipedia.org/', article_link)

    return first_link
test_url = "https://en.wikipedia.org/wiki/Theodore_Roosevelt"

print(test_url)
print(find_first_link(test_url))

https://en.wikipedia.org/wiki/Theodore_Roosevelt
/wiki/Help:Pronunciation_respelling_key
https://en.wikipedia.org/wiki/Asthma


### Write a program to follow first link on a wikipedia page to a target webpage.

In [34]:
import time
import urllib

import bs4
import requests


start_url = "https://en.wikipedia.org/wiki/Special:Random"
target_url = "https://en.wikipedia.org/wiki/Philosophy"

def find_first_link(url):
    response = requests.get(url)
    html = response.text
    soup = bs4.BeautifulSoup(html, "html.parser")

    # This div contains the article's body
    # (Note: Body nested in two div tags)
    content_div = soup.find(id="mw-content-text").find(class_="mw-parser-output")

    # stores the first link found in the article, if the article contains no
    # links this value will remain None
    article_link = None

    # Find all the direct children of content_div that are paragraphs
    for element in content_div.find_all("p", recursive=False):
        # Find the first anchor tag that's a direct child of a paragraph.
        # It's important to only look at direct children, because other types
        # of link, e.g. footnotes and pronunciation, could come before the
        # first link to an article. Those other link types aren't direct
        # children though, they're in divs of various classes.
        if element.find("a", recursive=False):
            article_link = element.find("a", recursive=False).get('href')
            if article_link.find("Help") <0:
                break

    if not article_link:
        return

    # Build a full url from the relative article_link url
    first_link = urllib.parse.urljoin('https://en.wikipedia.org/', article_link)

    return first_link

# Function to determine whether to continue webcrawl
def continue_crawl(search_history, target_url, max_steps=25):
    if search_history[-1] == target_url:
        print("We've found the target article!")
        return False
    elif len(search_history) > max_steps:
        print("The search has gone on suspiciously long, aborting search!")
        return False
    elif search_history[-1] in search_history[:-1]:
        print("We've arrived at an article we've already seen, aborting search!")
        return False
    else:
        return True

article_chain = [start_url]

while continue_crawl(article_chain, target_url):
    print(article_chain[-1])

    first_link = find_first_link(article_chain[-1])
    if not first_link:
        print("We've arrived at an article with no links, aborting search!")
        break

    article_chain.append(first_link)

    time.sleep(2) # Slow things down so as to not hammer Wikipedia's servers

print(article_chain[-1])
    

https://en.wikipedia.org/wiki/Special:Random
https://en.wikipedia.org/wiki/Japanese_people
https://en.wikipedia.org/wiki/Nation
https://en.wikipedia.org/wiki/Community
https://en.wikipedia.org/wiki/Norm_(social)
https://en.wikipedia.org/wiki/Sociology
https://en.wikipedia.org/wiki/Society
https://en.wikipedia.org/wiki/Social_group
https://en.wikipedia.org/wiki/Social_science
https://en.wikipedia.org/wiki/Discipline_(academia)
https://en.wikipedia.org/wiki/Branch_(academia)
https://en.wikipedia.org/wiki/Knowledge
https://en.wikipedia.org/wiki/Fact
https://en.wikipedia.org/wiki/Reality
https://en.wikipedia.org/wiki/Existence
https://en.wikipedia.org/wiki/Ontology
https://en.wikipedia.org/wiki/Philosophical
https://en.wikipedia.org/wiki/Greek_language
https://en.wikipedia.org/wiki/Modern_Greek
https://en.wikipedia.org/wiki/Colloquialism
https://en.wikipedia.org/wiki/Linguistics
https://en.wikipedia.org/wiki/Science
https://en.wikipedia.org/wiki/Latin_language
https://en.wikipedia.org/wiki

In [ ]:
import urllib.request, urllib.parse, urllib.error